# Analyzing spiking responses to visual stimuli

In this notebook, you will engage in some exploratory data analysis of one of Frank's recording sessions, the same one we used for our spike sorting exercise this morning.

This is a recording from area V2 in tree shrew visual cortex. Not a lot has been published about the neurons in this area. The recording contains responses to several kinds of visual stimuli as well as to electrical stimuli delivered to V1.

The point of the exercise is not to reach a scientific conclusion. (After all, this is *N* = 1 recording), but rather to try a few different analytic approaches and see if you can come up with some preliminary ideas about what this neurons in tree shrew V2 are all about. This exercise is intentionally open-ended. We encourage you to try alternative approaches.

## Accessing the data

First, we need to access the data. This morning, we already accessed the raw traces for this experiment, now we will also open the KiloSort 2.0 spike data. If you decided you like one of the other sorters better, you can absolutely use its results instead. (In the "Sorted" notebook, you can see how to extract the spike trains.)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/cheninstitutecaltech/Caltech_DATASAI_Neuroscience_23/blob/main/07_13_23_day4_adapting_preprocessing_data/code/diy_notebooks/colab/DataSAI_Wagenaar_Activity.ipynb)

In [ ]:
%pip install  ephysio
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ephysio import openEphysIO
from ephysio import kilosortIO
import numpy as np
import pickle
import json
import matplotlib.pyplot as plt

In [ ]:
root = "/content/drive/MyDrive/datasai-daw/data/2021-07-20_11-59-01"
oe = openEphysIO.Loader(root)

In [ ]:
oe.spikestreams()

In [ ]:
ks = kilosortIO.Reader(oe.contfolder(oe.spikestream(), rec=9) + "/kilosort20_output")

Frank stores metadata about each recording in JSON files with the experimental data. This is a very convenient way to make sure you don't have to rely on memory for knowing which binary channel in the recording carries which signal:

In [ ]:
with open(f"{root}/exptINFO_2021-07-20_11-59-01+9.json") as fd:
    info = json.load(fd)
info

We learn that the sync pulses for electrical stimuli can be found on digital channel 2, whereas the sync pulses for visual stimuli can be found on digital channel 4. We also see that there are two blocks of electrical stimuli, one of type "boatload", and one of type "ramptest". (Ask Frank to show you the detailed log files to see exactly what that means.) And we see that there were four blocks of visual stimuli. The record shows the names of the python scripts that controlled those stimuli, and the name of "pickle" files that store the detailed parameters. We will access those in a little bit.

Let's first extract the time stamps of the state transitions on the digital channels, so we can find out when exactly the stimuli occurred:

In [ ]:
evts = oe.nidaqevents(oe.spikestream(), rec=9)

Do we actually have channels 2 and 4?

In [ ]:
evts.keys()

Yes, we do.

Let's extract the timestamps of the electrical stimuli:

In [ ]:
eblks = oe.inferblocks(evts[2], oe.spikestream())
len(eblks)

We have two blocks. Very good. How many stimuli per block?

In [ ]:
[len(x) for x in eblks]

Looks plausible.

Same for the visual stimuli:

In [ ]:
vblks = oe.inferblocks(evts[4], oe.spikestream())
len(vblks)

Four blocks. Exactly as advertised.

In [ ]:
[len(x.flatten()) for x in vblks]

(Why does the counting of visual stimuli look different than of electrical stimuli? The answer is technical: The electrical stimuli are each marked by a short TTL pulse, whereas the visual stimuli are marked by a black-to-white or white-to-black transition of a little square on the corner of the display monitor, which gets picked up by a phototransistor. As a consequence, we only care about the 0-to-1 transitions of the electrical markers, but we care about both the 0-to-1 and the 1-to-0 transitions of the visual markers.)

To get you started, let's focus on the "gratings" stimulus block. This compromised multiple presentations of full-screen gratings, with 6 different orientations, 5 different spatial frequencies, and 4 different combinations of eye presentation (explained below).

The first step is to extract the stimulus time stamps, and to load the "pickle" that contains the relevant stimulus information.

In [ ]:
tframe = vblks[1].flatten()
fn = f"{root}/visual_stimuli/{info['visualstimuli']['1'][1]}"
with open(fn, 'rb') as fd:
    pkl = pickle.load(fd)

So what information do we have?

In [ ]:
pkl.keys()

By convention, variables with ALLCAPS names are constants. Let's educate ourselves.

In [ ]:
for k, v in pkl.items():
    if k==k.upper():
      print(f"{k}: {v}")

What this means, is that the images were presented at a rate of 10 frames per second, 9 repeats of each stimulus, total 120 different kinds of images, 3 s of gray at the beginning of the experiment, 6 orientations (0°, 30°, etc.), four spatial frequencies (0.1 cycles/degree, 0.2 cycles/degree, etc), one phase (i.e., Frank did not in this recording test whether translating the grating by a fraction of the period made a difference), and 4 "eye" combinations. These mean, respectively: image presented only to the right eye (1, contralateral to the recording site in left V2), only to the left eye (2, ipsilateral), to both eyes (3), or with a sensory conflict (-1). The sensory conflict meant that the right eye viewed the orientation specified, while the left eye viewed a stimulus that was 90° rotated.

How do we know which stimulus was presented when? That's encoded in the "osp_idx":

In [ ]:
len(pkl['osp_idx'])

In [ ]:
tstim = []
ospe = []
for k, x in enumerate(pkl['osp_idx']):
    if x:
        tstim.append(tframe[k])
        ospe.append(x)
    # Else, this is a gray frame, which we will ignore for now.
tstim = np.array(tstim)
ospe = np.array(ospe)

In [ ]:
tstim.shape

In [ ]:
ospe.shape

### Exercise

Do these numbers make sense? We should have 9 repeats of 6 orientations x 5 spatial frequencies x 4 eye combinations.

In [ ]:
# Insert your code here


What's in the `ospe`?

In [ ]:
ospe[:10]

Conclusion, the first column represents the orientation (0 ⇒ 0°, 1 ⇒ 30°, etc), the second the spatial frequency, the fourth the eye (0 ⇒ right, 1 ⇒ left, 2 ⇒ both, 3 ⇒ conflict). The third column is boring.

### Exercise

Look at the first few numbers in the `tstim` vector, and guess whether they represent time stamps in seconds, milliseconds, or samples.

In [ ]:
# Insert your code here


As we saw earlier, KiloSort reports both "good" clusters (putative single-neuron clusters) and "mua" clusters (multi-unit activity). We will only analyze the former, as the "mua"-labeled clusters very poorly matched the other spike sorters, so we we don't trust them.

In [ ]:
tspk = ks.spikesbycluster('good')

### Exercise

What is the container type of tspk? Why do you think the data are represented like that? What is the contained type? In what unit do you think spike times are stored?

In [ ]:
# Insert your code here


Let's convert all times to seconds, so we don't have to keep track of sampling rates from here on out.

In [ ]:
fshz = oe.samplingrate(oe.spikestream())
tstim_s = tstim / fshz
tspk_s = { k: v / fshz for k, v in tspk.items() }

You can absolutely write your own code to align spike times to stimulus times and count per-stimulus responses, but there are many libraries that provide this functionality, so there is probably no need to reinvent the wheel. If you already have a favorite library, by all means use it. Otherwise, here is mine:

In [ ]:
from ephysio.spikestats import SpikeStats

We feed it our stimulus times and dictionary of spike times:

In [ ]:
ss = SpikeStats(tstim_s, tspk_s)

And now we can extract the latencies of any particular neuron's spikes relative to the stimuli. For example, let's look when neuron #1 fired relative to our stimuli:

In [ ]:
lat, tri = ss.latencies(1, -100, 100)
plt.plot(lat, tri, '.')

### Exercise: Does this look right?

Why did we plot time from -100 ms to +100 ms? Why do you think there are fewer spikes during the last 200 stimuli compared to the first 200? What could explain that there are more spikes between -100 ms and -75 ms than between -50 ms and -25 ms?

Look at a couple other units as well, to get a better feel for the data. (Remember: don't walk around with a blindfold on.) What else would you like to do to explore the raw spike data?

In [ ]:
# Insert your code here


Let's count the number of spikes fired by neuron #1 in the first 100 ms after each stimulus:

In [ ]:
cnt = ss.spikecounts(1, 0, 100)
plt.plot(cnt, '.')

That cannot be right. The raster clearly showed plenty of spikes, yet we count nothing.

### Debugging time

Let's (for the moment) assume the library code is correct. Did we use it incorrectly?

In [ ]:
ss.spikecounts?

Looks like we used the function correctly. What else could be wrong? Did we feed broken data into SpikeStats? Let's take a look:

In [ ]:
plt.plot(tstim_s, '.')

The stimulus times look plausible (nicely sequential, correct number, spanning a reasonable amount of time).

What about the spike times?

In [ ]:
plt.plot(tspk_s[1], '.')

Those are not in order. Is that a problem?

In [ ]:
SpikeStats?

It doesn't say. That's what you get for using homebrew libraries. But it's a good guess. Let's try sorting the spike times before feeding them in.

In [ ]:
tspk_s = { k: np.sort(v) / fshz for k, v in tspk.items() }
ss = SpikeStats(tstim_s, tspk_s)

### Exercise

Did that solve the problem?

In [ ]:
# Insert your code here


OK, it did. So now we can extract the responses for each of the cells. For instance:

In [ ]:
cnts = {}
for celid in tspk_s.keys():
  cnts[celid] = ss.spikecounts(celid, 0, 100)
len(cnts), type(cnts[1]), cnts[1].shape

So now we have 1080 response counts for each of 228 cells, stored in a dictionary of numpy arrays.

## Reshaping the data for further processing

For the sake of this exercise, we will feed these data into PCA to see whether the population differentiated between the various gratings we presented to the animal. Let's construct a PCA instance and remind ourselves what shape of data it wants.

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(10)
pca.fit?

So we need to present the data as a single array. In our case, the “features” are the neurons, and the “samples” are the stimuli. (I always have to think twice about that, so let's keep in mind that we need to check that we are doing it right.)

At the moment, the data are in a dictionary, so we need to repackage them as a single big array.

### Exercise

Extract a vector of cell IDs from the dictionary and a CxK array of the K stimulus responses of each of the C cells.

In [ ]:
# Insert your code here
cellids = ...
resps = ...

Now, we can feed the responses straight into the PCA:

In [ ]:
pc = pca.fit_transform(resps)

### Exercise

What should the shape of the `pc` output be? Did we code that right? If not, can you fix it?

In [ ]:
### Insert your code here


Let's see if orientation is captured in the first two principal components.

In [ ]:
plt.figure()
for o in range(len(pkl["ORIENT"])):
  idx = np.nonzero(ospe[:,0]==o)[0]
  plt.plot(pc[idx,0], pc[idx,1], '.')

### Exercise

What do you think? Anything there?

Check out a few other PCs, perhaps?

In [ ]:
# Insert your code here


### Exercise

Is it a problem that we included the conflict stimuli? How can we avoid plotting those? Should we investigate left/right/both eye stimuli separately?

In [ ]:
# Insert your code here


### Exercise

Make a similar plot, but use color to represent the different spatial frequencies.

*Hint:* Loop `for s in range len(pkl["SPFREQ"])`

In *this* case, are conflict stimuli equally problematic?

In [ ]:
# Insert your code here


Do you agree that the population as a whole appears to respond differently to different spatial frequencies?

### Exercise

Make a simple bar plot of total population response as a function of spatial frequency. Do the results mesh with what the PCA indicated?

In [ ]:
# Insert your code here


## The end is the beginning

From here, you could explore many follow-up ideas. For instance:

* Are there subpopulations that respond strongly to spatial frequencies that the area as a whole responds only weakly to? Nonnegative matrix factorization might be a starting point for exploring that.

* Are the responsive neurons concentrated in certain layers of the cortex? You can ask, for any unit, how far from the tip of the probe it sits with `ks.tipdist_unit`.

* Do different subpopulations respond more strongly to stimuli presented to one or the other eye?

* What are *you* curious about?

In [ ]:
# Insert your code here...
